In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shapely.geometry import Polygon, LineString, Point
import ipywidgets as widgets
from ipywidgets import interact

import sys
# Add measurement mcts python package to path
sys.path.append('../src/measurement_mcts')

from measurement_mcts.utils.utils import angle_difference, min_max_normalize

In [16]:
def evaluate(state, bearing_wgt, dist_wgt, ocl_turn_wgt, ocl_dist_wgt, draw=False) -> float:
    """
    Evaluates a DecisionNode by using a convex combination of distance to closest OOI point and angle to OOI.
    
    :param state: (np.ndarray) the state of the car and OOI (position(0:2), corner means(2:10), corner covariances(10:74))
    :return: (float) the cumulative reward observed during the tree traversing.
    """
    #### Basic data extraction
    # Pull out the car position and yaw from the state
    car_state = state[0]
    car_pos = car_state[0:2]
    car_yaw = car_state[2]
    
    # Pull out the OOI corner positions from the state
    ooi_state = state[1]
    ooi_reshaped = np.reshape(ooi_state, (4, 2))
    
    # Pull out covariances of corners and get trace for each corner
    cov_state = state[2]
    cov_diag = np.diag(cov_state)
    corner_traces = np.zeros((int(cov_diag.shape[0]/2),))
    
    # Iterate through covariance diagonal elements
    for i in range(corner_traces.shape[0]):
        # Calculate trace of covariance matrix for each corner
        corner_traces[i] = cov_diag[i*2] + cov_diag[(i*2) + 1]
    

    #### Find what corners are occluded by front of object using shapely
    # Create line of sight from the car center to each corner of the OOI
    car_pt = Point(car_pos)
    lines_of_sight = [LineString([car_pos, Point(corner)]) for corner in ooi_reshaped]
    
    # Check if each line of sight cross the OOI (which means it is not observable)
    ooi_polygon = Polygon(ooi_reshaped)
    intersection_bools = [line.crosses(ooi_polygon) for line in lines_of_sight]
    
    
    #### Calculate squared distance and bearing to each corner
    # Calculate squared distance of Car to OOI points
    squared_dists = np.sum((ooi_reshaped - car_pos)**2, axis=1)
    
    # Create scaled cumulative distance, bearing, occlusion turn steps, and occlusion distance steps
    cum_dist = 0.
    cum_bearing = 0.
    cum_occlusion = 0.
    
    # Iterate through each corner trace
    for i in range(corner_traces.shape[0]):
        # If corner is occluded use closest observable point weighting
        if intersection_bools[i]:
            # Return car position projected to corner observable line
            extension_length = 100
            
            # Extend vector from corner to prev and next corners by subtracting tail, multiplying by extension and then adding tail back
            prev_corner_extended = (ooi_reshaped[i-1] - ooi_reshaped[i]) * extension_length + ooi_reshaped[i-1]
            next_corner_extended = (ooi_reshaped[(i+1)%4] - ooi_reshaped[i]) * extension_length + ooi_reshaped[(i+1)%4]
            
            # Make these into Line Strings to use shapely functions
            obs_line1 = LineString([ooi_reshaped[i], prev_corner_extended])
            obs_line2 = LineString([ooi_reshaped[i], next_corner_extended])
            
            # Project the car position onto the line, get length along line
            obs_len1 = obs_line1.project(car_pt)
            obs_len2 = obs_line2.project(car_pt)
            
            # Use length along line to get projected point
            obs_pt1 = obs_line1.interpolate(obs_len1)
            obs_pt2 = obs_line2.interpolate(obs_len2)
            
            # Draw the projected closest observation points
            if draw:
                self.ui.draw_arrow(car_pos, *obs_pt1.coords)
                self.ui.draw_arrow(car_pos, *obs_pt2.coords)
                
            # Convert into a numpy array
            obs_pts = np.array([*obs_pt1.coords, *obs_pt2.coords])
            
            # Iterate through each observable point to find the lowest cost point
            obs_pt_costs = np.zeros((obs_pts.shape[0],))
            for j, obs_pt in enumerate(obs_pts):
                ##### First weighting is based on the number of timesteps to reach the observable point
                
                # Start with the difference in angle between the car and direction to observe the corner (extension point to corner angle)
                yaw_err = angle_difference(car_yaw, np.arctan2(obs_pt[1] - ooi_reshaped[i][1], obs_pt[0] - ooi_reshaped[i][0]))
                
                # Subtract half of the car fov since we can see the corner if it is within half of the fov
                yaw_err -= np.deg2rad(45 / 2)
                
                # Make sure this is positive
                yaw_err = max(yaw_err, 0)
                
                # Find maximum angle car can turn in one timestep
                max_yaw = 10 * np.tan(np.deg2rad(45)) / 4
                
                # Get number of timesteps to turn to be able to observe the corner
                turn_steps = yaw_err / max_yaw
                
                #### Second weighting is based on the distance to the observable point
                # Find the squared distance to the observable point
                dist_err = np.sum((obs_pt - car_pos)**2)
                
                # Get the number of steps to reach that point
                dist_steps = np.sqrt(dist_err) / 10
                
                # print("Turn steps:", turn_steps, "Dist steps:", dist_steps)
                
                #### Normalize the turn and distance steps to a max of 10 steps
                turn_steps = min_max_normalize(turn_steps, 0, 10)
                dist_steps = min_max_normalize(dist_steps, 0, 10)
                
                # Add the weighted turn and distance steps to the observable point costs to later find the minimum
                obs_pt_costs[j] = ocl_turn_wgt * turn_steps + ocl_dist_wgt * dist_steps
                
            # Add the weighted minimum cost of the observable points to the cumulative occlusion
            cum_occlusion += corner_traces[i] * np.min(obs_pt_costs)
            
            print("Chosen observable point coords: ", obs_pts[np.argmin(obs_pt_costs)])
            print("Chosen observable point cost: ", np.min(obs_pt_costs))

            
        # Otherwise weight based on bearing and distance to corner
        else:
            # Find bearing to current corner
            ooi_bearing = np.arctan2(ooi_reshaped[i, 1] - car_pos[1], ooi_reshaped[i, 0] - car_pos[0])
            
            # Subtract the car yaw to get the relative bearing to the OOI point
            bearing_delta = abs(ooi_bearing - car_yaw)
            
            # Normalize both the distance and bearing to be between 0 and 1
            norm_dist = min_max_normalize(squared_dists[i], 0., 2000.)
            norm_bearing = min_max_normalize(bearing_delta, 0., np.pi)
            
            # Add covariance trace weighted bearing and squared distance to cumulative distance and bearing
            cum_dist += corner_traces[i] * norm_dist
            cum_bearing += corner_traces[i] * norm_bearing
    
    # Return weighted convex combination (alpha and beta add to 1) of cumulative covariance weighted distance and bearing
    # This is negative because we want to minimize the distance and bearing (punishment for being far away or off bearing)
    evaluation_value = -1 * (dist_wgt * cum_dist + bearing_wgt * cum_bearing + cum_occlusion)

    return evaluation_value

In [11]:
# Test the evaluation function
car_state = np.array([20., 20., 0.])
ooi_mean = np.array([54., 52., 54., 48., 46., 48., 46., 52.])
ooi_cov = np.diag([1., 1., 1., 1., 1., 1., 1., 1.])

state = (car_state, ooi_mean, ooi_cov)

evaluate(state)

Chosen observable point coords:  [54. 20.]
Chosen observable point cost:  0.3871238898038469


-7.402022194705203

In [22]:
# Plot the car and OOI interactively
car_x = widgets.IntSlider(value=20, min=0, max=100, step=1, description='Car X:', continuous_update=False)
car_y = widgets.IntSlider(value=20, min=0, max=100, step=1, description='Car Y:', continuous_update=False)
car_yaw = widgets.IntSlider(value=0, min=0, max=360, step=1, description='Car Yaw:', continuous_update=False)
dist_weight = widgets.FloatSlider(value=0.5, min=0., max=1., step=0.1, description='Distance Weight:', continuous_update=False)
bearing_weight = widgets.FloatSlider(value=0.5, min=0., max=1., step=0.1, description='Bearing Weight:', continuous_update=False)
ocl_turn_weight = widgets.FloatSlider(value=0.5, min=0., max=1., step=0.1, description='Occlusion Turn Weight:', continuous_update=False)
ocl_dist_weight = widgets.FloatSlider(value=0.5, min=0., max=1., step=0.1, description='Occlusion Distance Weight:', continuous_update=False)

def plot(car_x, car_y, car_yaw, bearing_weight, dist_weight, ocl_turn_weight, ocl_dist_weight):
    car_state = np.array([car_x, car_y, car_yaw])
    ooi_mean = np.array([54., 52., 54., 48., 46., 48., 46., 52.])
    ooi_cov = np.diag([1., 1., 1., 1., 1., 1., 1., 1.])

    state = (car_state, ooi_mean, ooi_cov)
    
    eval = evaluate(state, bearing_weight, dist_weight, ocl_turn_weight, ocl_dist_weight)
    print("Evaluation: ", eval)
    
    # Plot the car and OOI
    fig, ax = plt.subplots()
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)
    
    # Plot the car
    car = patches.Rectangle((car_x - 1, car_y - 1), 2, 2, angle=car_yaw, color='blue')
    ax.add_patch(car)
    
    # Plot the OOI
    ooi = patches.Polygon(ooi_mean.reshape((4, 2)), closed=True, fill=True, color='red')
    ax.add_patch(ooi)
    
    plt.show()
    
interact(plot, car_x=car_x, car_y=car_y, car_yaw=car_yaw, bearing_weight=bearing_weight, dist_weight=dist_weight, ocl_turn_weight=ocl_turn_weight, ocl_dist_weight=ocl_dist_weight)

interactive(children=(IntSlider(value=20, continuous_update=False, description='Car X:'), IntSlider(value=20, …

<function __main__.plot(car_x, car_y, car_yaw, bearing_weight, dist_weight, ocl_turn_weight, ocl_dist_weight)>